In [1]:
!nvidia-smi

Sat Sep 21 13:22:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   37C    P8             15W /   40W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#@title Clone repo
!git clone https://github.com/soumik-kanad/diff2lip.git

In [ ]:
#@title Install requirements
import os
os.chdir('diff2lip')
!pip install -r requirements.txt

In [ ]:
#@title Download Checkpoint
! mkdir -p checkpoints
! gdown 15vMcyPtFDEttNwelP3IouwsMpnccTqit -O checkpoints/checkpoint.pt

In [ ]:
#@title helper functions
from google.colab import files
import shutil
os.makedirs("data", exist_ok=True)

def upload_file():
    uploaded = files.upload()
    uploaded_paths = []
    for filename in uploaded.keys():
        dst_path = os.path.join("data", filename)
        print(f'move {filename} to {dst_path}')
        shutil.move(filename, dst_path)
        uploaded_paths.append(dst_path)
    return uploaded_paths

In [ ]:
#@title Upload Video
# from sentence_generate import narrative
# v = upload_file()
video_path="test-video.mp4"
# audio_path=v[0]

In [ ]:
#@title Upload Audio (Upload again or skip this cell if the audio and video are present in the same video uploaded above)
# a = upload_file()
audio_path="video_narrative.mp3"

In [ ]:
#@title Set these paths manually if you upload your files manually
# video_path="data/Harry_potter_hp_spanish_trim.mp4"
# audio_path="data/Harry_potter_hp_spanish_trim.mp4"
out_path="output.mp4"

In [ ]:
#@title Setting Flags
sample_mode="cross" # or "reconstruction"
generate_from_filelist=0
model_path="checkpoints/checkpoint.pt"
pads="0,0,0,0"

#cross vs reconstruction
if sample_mode == "reconstruction":
    sample_input_flags="--sampling_input_type=first_frame --sampling_ref_type=first_frame"
elif sample_mode == "cross":
    sample_input_flags="--sampling_input_type=gt --sampling_ref_type=gt"
else:
    print('Error: sample_mode can only be \"cross\" or \"reconstruction\"')

MODEL_FLAGS=f"--attention_resolutions 32,16,8 --class_cond False --learn_sigma True --num_channels 128 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm False"
DIFFUSION_FLAGS=f"--predict_xstart False  --diffusion_steps 1000 --noise_schedule linear --rescale_timesteps False"
SAMPLE_FLAGS=f"--sampling_seed=7   {sample_input_flags} --timestep_respacing ddim25 --use_ddim True --model_path={model_path}"
DATA_FLAGS=f"--nframes 5 --nrefer 1 --image_size 128 --sampling_batch_size=32 "
TFG_FLAGS=f"--face_hide_percentage 0.5 --use_ref=True --use_audio=True --audio_as_style=True"
GEN_FLAGS=f"--generate_from_filelist {generate_from_filelist}  --video_path={video_path} --audio_path={audio_path} --out_path={out_path} --save_orig=False --face_det_batch_size 16 --pads {pads} --is_voxceleb2=False"



In [ ]:
#@title Run Generation (took ~ 4 mins for the example video)
!python generate.py {MODEL_FLAGS}  {DIFFUSION_FLAGS}  {SAMPLE_FLAGS} {DATA_FLAGS} {TFG_FLAGS} {GEN_FLAGS}

In [ ]:
#@title Display Output Video
import moviepy.editor
moviepy.editor.ipython_display("output.mp4")